In [27]:
import os
import sys
sys.path.append(os.getcwd())
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.io.wavfile import read
from glob import glob
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image


In [28]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2024-11-17 23:48:14.320459: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 23:48:14.320629: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 23:48:14.320674: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 23:48:14.320909: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 23:48:14.320922: I tensorflow/core/common_runtime/gpu/gpu

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9597903527511096657
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10650386432
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11844495622297161802
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [29]:
ip_size = 96000

def preprocess(data_path):
   try:
      array = read(data_path)[1]
      array = array.astype(np.float32)
      if array.ndim >= 2:
         array = np.mean(array, axis=1)
      if array.shape[0] > ip_size:
         array = array[:ip_size]
         return array
      else:
         array = np.append(array, np.zeros(96000-array.shape[0]))
         pass
      return array
   except ValueError:
      pass


data_dir = os.path.join(os.getcwd().rsplit("/",1)[0],"origin_data")
animal_type = "dog"
# state_list = [dir for dir in os.listdir(os.path.join(data_dir,animal_type))]
state_list = ["hostile","relax","whining"]

X_train = []
y_train = []
for state in state_list:
   state_data = glob(os.path.join(data_dir,animal_type,state,"*.wav"))
   for d_path in state_data:
      if not isinstance(preprocess(d_path), np.ndarray):
         continue
      zero = np.zeros(3)
      zero[state_list.index(state)] = 1
      X_train.append(preprocess(d_path))
      y_train.append(zero)
      
X_train = np.array(X_train)
y_train = np.array(y_train)

valid_rate = 0.1
lenX = len(X_train)

random_batch = np.random.choice(range(lenX), int(lenX*valid_rate), replace=False)
random_batch

X_val = X_train[random_batch]
y_val = y_train[random_batch]

X_train = np.delete(X_train, random_batch,axis=0)
y_train = np.delete(y_train, random_batch,axis=0)



In [30]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((278, 96000), (278, 3), (30, 96000), (30, 3))

In [31]:
from WaveNetClassifier import WaveNetClassifier
# from https://github.com/mjpyeon/wavenet-classifier.git

epoch = 5
wnc = WaveNetClassifier((ip_size,), (3,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 original_input (InputLayer  [(None, 96000)]              0         []                            
 )                                                                                                
                                                                                                  
 reshaped_input (Reshape)    (None, 96000, 1)             0         ['original_input[0][0]']      
                                                                                                  
 dilated_conv_1 (Conv1D)     (None, 96000, 40)            120       ['reshaped_input[0][0]']      
                                                                                                  
 dilated_conv_2_tanh (Conv1  (None, 96000, 40)            3240      ['dilated_conv_1[0][0]']

In [34]:
wnc.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = epoch, batch_size = 4, optimizer='adam', save=True, save_dir='./')

Epoch 1/5
70/70 [==============================] - ETA: 0s - loss: 1.1938 - accuracy: 0.5683
Epoch 1: val_loss improved from inf to 0.87647, saving model to ./saved_wavenet_clasifier.h5
70/70 [==============================] - 49s 524ms/step - loss: 1.1938 - accuracy: 0.5683 - val_loss: 0.8765 - val_accuracy: 0.6667
Epoch 2/5


/home/ddust/anaconda3/envs/furry/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


70/70 [==============================] - ETA: 0s - loss: 0.9110 - accuracy: 0.6403
Epoch 2: val_loss improved from 0.87647 to 0.66158, saving model to ./saved_wavenet_clasifier.h5
70/70 [==============================] - 33s 476ms/step - loss: 0.9110 - accuracy: 0.6403 - val_loss: 0.6616 - val_accuracy: 0.7667
Epoch 3/5
70/70 [==============================] - ETA: 0s - loss: 0.7818 - accuracy: 0.6906
Epoch 3: val_loss did not improve from 0.66158
70/70 [==============================] - 33s 476ms/step - loss: 0.7818 - accuracy: 0.6906 - val_loss: 0.7777 - val_accuracy: 0.7667
Epoch 4/5
70/70 [==============================] - ETA: 0s - loss: 0.7256 - accuracy: 0.7014
Epoch 4: val_loss improved from 0.66158 to 0.48936, saving model to ./saved_wavenet_clasifier.h5
70/70 [==============================] - 31s 445ms/step - loss: 0.7256 - accuracy: 0.7014 - val_loss: 0.4894 - val_accuracy: 0.8667
Epoch 5/5
65/70 [==========================>...] - ETA: 2s - loss: 0.7161 - accuracy: 0.7308
E

In [ ]:
# Epoch 4/5
# 70/70 [==============================] - ETA: 0s - loss: 0.7256 - accuracy: 0.7014
# Epoch 4: val_loss improved from 0.66158 to 0.48936, saving model to ./saved_wavenet_clasifier.h5
# 70/70 [==============================] - 31s 445ms/step - loss: 0.7256 - accuracy: 0.7014 - val_loss: 0.4894 - val_accuracy: 0.8667
# Epoch 5/5
# 65/70 [==========================>...] - ETA: 2s - loss: 0.7161 - accuracy: 0.7308
# Epoch 5: val_loss did not improve from 0.48936
# 70/70 [==============================] - 33s 478ms/step - loss: 0.7231 - accuracy: 0.7158 - val_loss: 0.5588 - val_accuracy: 0.8000